In [17]:
import indicoio
import operator
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from textblob import TextBlob


indicoio.config.api_key = 'c838a3547516c7e03ebf4a16839af30e'
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [15]:
nltk.download('brown')

[nltk_data] Downloading package brown to /Users/jxu5/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [4]:
df = pd.DataFrame(pd.read_csv("Verbatims_data2.csv"))

In [5]:
df

,BU,OII Related,RateOfOII,Login Related Flag,Number of Records,OFFERING,RATING,REVIEW_TEXT,app_identifier,app_store,bank_flag,count,review_date,score,tag,title
0,CTG,0,0,False,1,My Refund,5,Great app,com.intuit.mobile.mytaxrefund,Google,0,nan,2/10/15,0,RATING ROLLUP,Great
1,CTG,0,0,False,1,TurboTax,5,Great,com.intuit.turbotax,Apple,0,nan,1/17/17,0,RATING ROLLUP,Great
2,SBG,0,0,False,1,QB Self Employed,3,Please include a way to clear all notification...,com.intuit.qbse,Apple,0,nan,6/27/17,0,RATING ROLLUP,Notifications!!!
3,CTG,0,0,False,1,TurboTax,1,I have been working on a efile for 9 days th...,com.intuit.turbotax.mobile,Google,0,nan,3/8/16,0,RATING ROLLUP,None
4,CTG,0,0,False,1,TurboTax,1,It now forces you to pay 30 dollars if you wa...,com.intuit.turbotax.mobile,Google,0,nan,1/20/17,0,RATING ROLLUP,None
5,CEG,0,0,False,1,Mint,5,"Love it, keeps me organized and simple to use",com.mint.internal,Apple,0,nan,1/3/16,0,RATING ROLLUP,Love it
6,CEG,0,0,False,1,Mint Bills,4,"By far my favorite financial app, makes everyt...",com.pageonce.ionce,Apple,0,nan,5/23/15,0,RATING ROLLUP,Love it
7,SBG,0,0,False,1,QBMobile,5,I used MYOB for years and naturally moved to e...,com.intuit.QBOiPad,Apple,0,nan,10/9/16,0,RATING ROLLUP,So much better than MYOB
8,CEG,0,0,False,1,Mint,5,I love the reminders when bills are coming due.,com.netgate,Google,1,nan,6/30/14,0,RATING ROLLUP,Great help
9,CEG,0,0,False,1,Mint,3,"Looks promising but a number of my accounts, t...",com.mint,Google,1,nan,1/6/18,0,RATING ROLLUP,None


In [8]:
df = df[["OFFERING","review_date","app_store","title","REVIEW_TEXT","RATING"]]
df.columns = ["offering","review_date","app_store","title","review","rating"]
df=df[df.review.str.len() > 5]

In [10]:
df.head()

,offering,review_date,app_store,title,review,rating
0,My Refund,2/10/15,Google,Great,Great app,5
2,QB Self Employed,6/27/17,Apple,Notifications!!!,Please include a way to clear all notification...,3
3,TurboTax,3/8/16,Google,None,I have been working on a efile for 9 days th...,1
4,TurboTax,1/20/17,Google,None,It now forces you to pay 30 dollars if you wa...,1
5,Mint,1/3/16,Apple,Love it,"Love it, keeps me organized and simple to use",5


In [11]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [12]:
def clean_text(text, remove_stopwords = True):
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text,  
                  flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = stopwords.words("english")
        stops.append("'s")
        stops.append("'t")
        stops.append("i'm")
        stops.append("i've")
        stops.append("app")
        text = [w for w in text if not w in stops]
        text = " ".join(text)
    return text

In [13]:
clean_reviews=[]

for i in df.review:
    clean_reviews.append(clean_text(i, remove_stopwords=True))
    
df['clean_reviews']=clean_reviews

In [16]:
phrases=[]

for i in df["clean_reviews"]:
    try:
        phrases.append(TextBlob(str(i)).noun_phrases[0])
    except:
        phrases.append("None")
        
df["phrases"]=phrases

In [18]:
sentiment_calls=[]
subset=int(len(df)/5)

for i in range(0,len(df),subset):
    sentiment_calls.append(indicoio.sentiment(list(df.iloc[i:i+subset]['review'])))


In [19]:
sentiments=[*sentiment_calls[0],*sentiment_calls[1],*sentiment_calls[2],*sentiment_calls[3],*sentiment_calls[4]]

In [20]:
polarity=[]
for i in sentiments:       
    
    if i<0.02:
        polarity.append('very negative')
    elif 0.02<=i and i<0.2:
        polarity.append('negative')
    elif 0.2<=i and i<0.8:
        polarity.append('neutral')
    elif 0.8<=i and i<0.98:
        polarity.append('positive')
    else:
        polarity.append('very positive')
        
df['polarity']=polarity

In [21]:
df.head(100)

,offering,review_date,app_store,title,review,rating,clean_reviews,phrases,polarity
0,My Refund,2/10/15,Google,Great,Great app,5,great,None,very positive
2,QB Self Employed,6/27/17,Apple,Notifications!!!,Please include a way to clear all notification...,3,please include way clear notifications could p...,clear notifications,negative
3,TurboTax,3/8/16,Google,None,I have been working on a efile for 9 days th...,1,working efile 9 days first time used years ago...,great ready punch screen,neutral
4,TurboTax,1/20/17,Google,None,It now forces you to pay 30 dollars if you wa...,1,forces pay 30 dollars want e file requires las...,e file,very negative
5,Mint,1/3/16,Apple,Love it,"Love it, keeps me organized and simple to use",5,love keeps organized simple use,simple use,very positive
6,Mint Bills,5/23/15,Apple,Love it,"By far my favorite financial app, makes everyt...",4,far favorite financial makes everything easier...,None,very positive
7,QBMobile,10/9/16,Apple,So much better than MYOB,I used MYOB for years and naturally moved to e...,5,used myob years naturally moved essentials onl...,myob years,positive
8,Mint,6/30/14,Google,Great help,I love the reminders when bills are coming due.,5,love reminders bills coming due,reminders bills,positive
9,Mint,1/6/18,Google,None,"Looks promising but a number of my accounts, t...",3,looks promising number accounts particularly o...,promising number accounts,very negative
10,Mint,7/13/17,Apple,Best App for Managing Finances,Have been using Mint for a while now - concer...,5,using mint concerned first giving financial in...,financial info,very positive


In [22]:
def banking(text):
    
    terms=['bank','banks','transaction','transactions','bill','bills','biller'
           ,'account','accounts','financial','balance','balances','categorize','categories','FI']

        
    if True:
        text = text.split()
        new_text = []
        
        x=0
        for word in text:
            if word in terms:
                x=x+1
                break
        new_text.append(x)

    return new_text

In [23]:
flags=[]
for i in df.clean_reviews:
    flags.append(banking(i)[0])
df['bank_flag']=flags

In [24]:
df.head(100)

,offering,review_date,app_store,title,review,rating,clean_reviews,phrases,polarity,bank_flag
0,My Refund,2/10/15,Google,Great,Great app,5,great,None,very positive,0
2,QB Self Employed,6/27/17,Apple,Notifications!!!,Please include a way to clear all notification...,3,please include way clear notifications could p...,clear notifications,negative,0
3,TurboTax,3/8/16,Google,None,I have been working on a efile for 9 days th...,1,working efile 9 days first time used years ago...,great ready punch screen,neutral,0
4,TurboTax,1/20/17,Google,None,It now forces you to pay 30 dollars if you wa...,1,forces pay 30 dollars want e file requires las...,e file,very negative,0
5,Mint,1/3/16,Apple,Love it,"Love it, keeps me organized and simple to use",5,love keeps organized simple use,simple use,very positive,0
6,Mint Bills,5/23/15,Apple,Love it,"By far my favorite financial app, makes everyt...",4,far favorite financial makes everything easier...,None,very positive,1
7,QBMobile,10/9/16,Apple,So much better than MYOB,I used MYOB for years and naturally moved to e...,5,used myob years naturally moved essentials onl...,myob years,positive,0
8,Mint,6/30/14,Google,Great help,I love the reminders when bills are coming due.,5,love reminders bills coming due,reminders bills,positive,1
9,Mint,1/6/18,Google,None,"Looks promising but a number of my accounts, t...",3,looks promising number accounts particularly o...,promising number accounts,very negative,1
10,Mint,7/13/17,Apple,Best App for Managing Finances,Have been using Mint for a while now - concer...,5,using mint concerned first giving financial in...,financial info,very positive,1


In [17]:
df.to_csv("10percent.csv")